In [1]:
from pathlib import Path
import sys
# Поднимаемся на два уровня вверх (из notebooks/ в InteriorClass/)
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))  # Теперь Python увидит src/

from src.config import RANDOM_SEED, SPLIT_RATIO, MIN_VAL_TEST_PER_CLASS, CLASS_LABELS
from src.dataset.splitter import DatasetSplitter
from src.dataset.interior_dataset import InteriorDataset, get_transforms
from src.models.interior_classifier_EfficientNet_B3 import InteriorClassifier
from tqdm import tqdm
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

In [2]:
# 1. Собираем все пути
current_dir = Path.cwd()
data_dir = current_dir.parent.parent / "data"
print(f"data_dir: {data_dir}")

dataset_dir = data_dir / "interior_dataset"

samples = InteriorDataset.collect_samples(dataset_dir=dataset_dir)

data_dir: /home/little-garden/CodeProjects/InteriorClass/data


In [3]:
print(len(samples))
print(samples[0])

55413
(PosixPath('/home/little-garden/CodeProjects/InteriorClass/data/interior_dataset/A0/A0_00000_f565.jpg'), 0)


In [4]:
# Создание сплиттера
splitter = DatasetSplitter(
    class_labels=["A0", "A1", "B0", "B1", "C0", "C1", "D0", "D1"],
    split_config=SPLIT_RATIO,
    random_seed=RANDOM_SEED
)

# Разделение данных
train_samples, val_samples, test_samples = splitter.split(samples, shuffle=True)

In [5]:
print(len(train_samples))
print(len(val_samples))
print(len(test_samples))

48988
3207
3218


In [6]:
# Конфигурация
batch_size = 64
epochs = 25
lr = 3e-5
img_size = 380  # Для EfficientNet-B3

# Datasets
train_dataset = InteriorDataset(
    train_samples,
    transform=get_transforms(mode='train'),
    mode='train'
)

val_dataset = InteriorDataset(
    val_samples,
    transform=get_transforms(mode='val'),
    mode='val'
)

test_dataset = InteriorDataset(
    test_samples,
    transform=get_transforms(mode='test'),
    mode='test'
)


# DataLoaders
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [7]:
def train():    
    # Модель и оптимизатор
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = InteriorClassifier(num_classes=len(InteriorDataset.CLASSES)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    # Цикл обучения
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0.0
        
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
        
        scheduler.step()
        
        # Валидация
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        # Метрики
        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(val_loader.dataset)
        report = classification_report(
            all_labels, all_preds, 
            target_names=InteriorDataset.CLASSES,
            zero_division=0
        )
        
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        print(report)
    
    # Финальная оценка на тестовом наборе
    model.eval()
    test_preds, test_labels = [], []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())
    
    print("\nFinal Test Results:")
    print(classification_report(
        test_labels, test_preds,
        target_names=InteriorDataset.CLASSES,
        digits=4
    ))


In [8]:
train()

Epoch 1/25:   0%|          | 0/765 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.53 GiB of which 5.06 MiB is free. Including non-PyTorch memory, this process has 23.50 GiB memory in use. Of the allocated memory 23.02 GiB is allocated by PyTorch, and 38.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)